In [ ]:
llm = LLM(model="ollama/llama3.2:1b", temperature=0.7,
          base_url="http://localhost:11434")

In [ ]:
# Working on structured output 
from crewai import Agent, Task, Crew, Process

agent = Agent(
    role="Senior Linguist",
    goal="Analyse the query and extract entity-relation-entity triplets",
    backstory="You are a senior linguist that is known for your analytical skills.",
    verbose=True,
    llm=llm,
)
#Pydantic schema
# 
from pydantic import BaseModel, Field 

class EntityRelationTriplet(BaseModel):
    entity1: str = Field("First entity in the relationship")
    relation: str = Field("Relationship between the entities")
    entity2: str = Field("Second entity in the relationship")




task = Task(
    description="""Analyse the query and return structured JSON
                   output in the form of
                   - entity
                   - relation
                   - entity

                   The query is: {query}
                   """,
    expected_output="A structured JSON object with the entity-relation-entity triplets",
    output_pydantic=EntityRelationTriplet,
    verbose=True,
    agent=agent
)

crew = Crew(
    agents=[agent],
    tasks=[task],
    process=Process.sequential,
    verbose=True
)

crew.kickoff(inputs={"query": "Paris is the capital of France."})


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 17f7dde9-072f-4dad-a059-21e6fb3b7796                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Linguist                                                                                         │
│                                                                                                                 │
│  Task: Analyse the query and return structured JSON                                                             │
│                     output in the form of                                                                       │
│                     - entity                                                                                    │
│                     - relation                                                                                  │
│                     - entity                                                                                    │
│                                                                                                                 │
│                     The query is: What is the capital of France?                                                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Linguist                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```json                                                                                                        │
│  {                                                                                                              │
│    "France": {                                                                                                  │
│      "capital": "Paris"                                                                                         │
│    }                                                                                                            │
│  }                                                                                                              │
│  ```                                                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: f608dae6-504c-4a94-bec7-3beead725713                                                                     │
│  Agent: Senior Linguist                                                                                         │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 17f7dde9-072f-4dad-a059-21e6fb3b7796                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ```json                                                                                          │
│  {                                                                                                              │
│    "France": {                                                                                                  │
│      "capital": "Paris"                                                                                         │
│    }                                                                                                            │
│  }                                                                                                              │
│  ```                                                                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

CrewOutput(raw='```json\n{\n  "France": {\n    "capital": "Paris"\n  }\n}\n```', pydantic=EntityRelationTriplet(entity1='First entity in the relationship', relation='Relationship between the entities', entity2='Second entity in the relationship'), json_dict=None, tasks_output=[TaskOutput(description='Analyse the query and return structured JSON\n                   output in the form of\n                   - entity\n                   - relation\n                   - entity\n\n                   The query is: What is the capital of France?\n                   ', name=None, expected_output='A structured JSON object with the entity-relation-entity triplets', summary='Analyse the query and return structured JSON\n   ...', raw='```json\n{\n  "France": {\n    "capital": "Paris"\n  }\n}\n```', pydantic=EntityRelationTriplet(entity1='First entity in the relationship', relation='Relationship between the entities', entity2='Second entity in the relationship'), json_dict=None, agent='Senior Lingu

In [ ]:
# Making the Crew return the output 

from pydantic import BaseModel, Field
from typing import List, Type

class InsightOutline(BaseModel):
    heading: str = Field( 
        description="The title of the outline section, serving as a main heading."
    )
    
    bullet_points: List[str] = Field(
        description="A list of key points summarizing this section."
    )

class Outline(BaseModel):
    sections: List[InsightOutline] = Field(
        description="A list of outline sections defining the article structure."
    )

class ArticleSection(BaseModel):
    title: str = Field(
        description="Markdown level-2 heading (e.g., '## Section Title')."
    )
    content: str = Field(
        description="Markdown-formatted prose for this section."
    )

class Article(BaseModel):
    sections: List[ArticleSection] = Field(
        description="Ordered list of the article sections with their titles and content."
    )



In [ ]:
from crewai import Crew, Task, Agent,Task, Process,LLM
from crewai_tools import SerperDevTool
import os
from dotenv import load_dotenv

load_dotenv()

web_search_tool = SerperDevTool(n_results=3,api_key=os.getenv("SERPER_API_KEY"))



researcher = Agent(
    role="Content Researcher",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory=("You're working on planning a blog article about the topic: {topic}. "
               "You collect information from the web and plan an outline " 
               "for an article that helps the audience learn something "
               "and make informed decisions. Your work is the basis for "
               "the Content Writer to write an article on this topic. "
               "Use the search tool to search for the most relevant and " 
               "factually accurate data about {topic}."),
    tools=[web_search_tool]
)

writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate article about the topic: {topic}",
    backstory=("You're working on writing a new blog post about the topic: {topic}. "
               "You base your writing on the work of the Content Researcher, "
               "who provides an outline and relevant context about the topic. "
               "You follow the main objectives and direction of the outline, "
               "as provided by the Content Researcher. You provide objective and "
               "impartial insights and back them up with information "
               "provided by the Content Researcher. "),
)

editor = Agent(
    role="Content Editor",
    goal=("Edit a given blog post to align with a structured "
          "format and remove false and misleading information. "
          "Ensure all the information provided is factually correct. "
          "Use the search tool for finding up-to-date and correct information."),
    backstory=("You are an editor who receives a blog post from the Content Writer. "
               "Your goal is to review the blog post to ensure that it follows "
               "best practices and adheres to a structured format. "
               "Verify the accuracy of information from the web using the "
               "search tool ensuring only accurate information is published."),
    tools=[web_search_tool]
)


plan_task = Task(
    description=("1. Prioritize the latest trends, "
                 "and noteworthy news on {topic}.\n"
                 "2. Identify the target audience, considering "
                 "their interests and pain points.\n"
                 "3. Develop a detailed content outline including "
                 "an introduction, key points, conclusion, and a call to action."
                 "4. Extract relevant data and key insights from multiple sources."),
    expected_output=("A comprehensive content plan document "
                     "with an outline, key insights and source references."),
    agent=researcher,
    tools=[web_search_tool],
    output_pydantic=Outline
)


write_task = Task(
    description=("1. Use the content plan to craft a compelling "
                 "blog post on {topic}.\n"
                 "2. Ensure sections are properly named and structured.\n"
                 "3. Ensure the post is structured with an "
                 "engaging introduction, insightful body, "
                 "and a summarizing conclusion.\n"),
    expected_output=("A well-written and well-structured blog post "
                    "with the most important and accurate "
                    "insights on {topic}. Each section should have "
                    "2 or 3 paragraphs. "),
    agent=writer,
    output_pydantic=Article
)

edit_task = Task(
    description=("Proofread the given blog post for "
                 "fact checking and grammatical errors. "
                 "Verify the accuracy of information and "
                 "cross-check facts with reliable sources "
                 "using the search tool and correct errors."),
    expected_output=("A well-written blog post, ready for publication, "
                     "each section should have 2 or 3 paragraphs."),
    agent=editor,
    tools=[web_search_tool],
    output_pydantic=Article
)

crew = Crew(
    agents=[researcher, writer, editor],
    tasks=[plan_task, write_task, edit_task],
    process=Process.sequential,
    verbose=True
)

TOPIC = "the future of space exploration"

result = crew.kickoff(inputs={"topic": TOPIC})
